<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=54b71a13329e280072a3b115df0461d9e97a29119f9cac9c776ad843b2569452
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7


Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-25 13:17:36
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.39 C
-------------------
Today PnL: -56.47 K (-0.4%)
Current PnL: -20.47 L (-13.77%)
CY Booked + Current PnL: -9.03 L (-6.08%)
-------------------
Total profit:  1.34 L
Total loss:  -21.81 L
-------------------
Total Booked + Current PnL: 17.95 L (14.75%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.22%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 86.34 L (62.06%)
Deployed:  1.22 C
Current:  1.39 C
CAGR/XIRR %: 8.15%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,13.31,59.0,H-MC,3.86,195490.0,30820.0,12511.0,0.55,18.72,6.40,26.31,89.0,2.46,1.41,28.19,X40N,NTT,AC
77,TTKPRESTIG,770.00,97.87,43.0,M-SC,2.89,85216.0,-15561.0,15654.0,0.48,-15.44,18.37,0.09,245.0,-0.99,0.61,11.95,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-45.32,54.0,H-MC,10.43,108920.0,8416.0,20695.0,-0.28,8.37,19.00,28.96,79.0,0.41,0.78,45.19,MH,ATH,METALS
84,VOLTAS,1530.00,-1.50,47.0,H-MC,3.07,206550.0,14808.0,22948.0,0.34,7.72,11.11,19.69,99.0,0.65,1.49,15.05,XY25,NTT,AC
57,RELIANCE,1533.00,-14.04,43.0,H-LC,4.29,214734.0,4368.0,24415.0,-0.47,2.08,11.37,13.68,37.0,0.18,1.54,18.94,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,0.96,43.0,H-SC,9.20,120456.0,-4841.0,63769.0,2.75,-3.86,52.94,47.03,141.0,-0.08,0.87,50.82,MH,ATH,POWER
61,SAMMAANCAP,326.00,-171.59,57.0,M-SC,5.50,82662.0,-19548.0,112941.0,1.83,-19.13,136.63,91.37,208.0,-0.17,0.59,34.19,XY25,NTT,FINANCE
32,HINDZINC,730.22,22.76,60.0,M-LC,2.43,200296.0,-4780.0,117353.0,1.67,-2.33,58.59,54.89,52.0,-0.04,1.44,21.20,X5K,ATH,METALS
8,AWL,485.00,-65.49,48.0,H-MC,3.29,245435.0,-56316.0,217750.0,1.66,-18.66,88.72,53.50,116.0,-0.26,1.76,9.01,XY24,NTT,FMCG
52,PGHH,17973.08,-28.56,61.0,H-MC,3.49,206625.0,5805.0,62979.0,1.04,2.89,30.48,34.25,80.0,0.09,1.49,8.24,X40,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-44.33,33.0,M-SC,1.47,94012.0,2514.0,76234.0,-3.28,2.75,81.09,86.07,223.0,0.03,0.68,36.52,XR,NTT,DURABLES
71,TANLA,1963.11,-17.45,63.0,H-SC,16.30,221804.0,-48957.0,378908.0,-3.21,-18.08,170.83,121.86,133.0,-0.13,1.59,73.51,AR,ATH,IT
55,RAJOOENG,143.10,-41.82,26.0,H-SC,12.73,88100.0,-14400.0,55001.0,-2.92,-14.05,62.43,39.61,114.0,-0.26,0.63,0.00,AR,ATH,MISC
73,TATAMOTORS,1065.00,-23.48,37.0,M-LC,2.28,241068.0,-35455.0,145533.0,-2.76,-12.82,60.37,39.81,54.0,-0.24,1.73,15.53,XY24,NTT,AUTO
5,ASIANPAINT,3465.66,-16.46,33.0,H-LC,7.35,210716.0,-41052.0,94253.0,-2.55,-16.31,44.73,21.13,27.0,-0.44,1.52,13.34,X40,ATH,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.8,41.0,H-MC,2.14,134549.0,-1239.0,43998.0,-0.22,-0.91,32.70,31.48,107.0,-0.03,0.97,11.10,X40,ATH,INSURANCE
23,DABUR,735.0,-7.8,39.0,H-MC,6.85,249508.0,1440.0,106964.0,-0.65,0.58,42.87,43.70,102.0,0.01,1.79,15.11,XY24,BTT,FMCG
86,WIPRO,420.0,-14.7,38.0,M-LC,5.50,149990.0,-955.0,110408.0,-1.09,-0.63,73.61,72.51,53.0,-0.01,1.08,5.35,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,51.0,H-SC,9.94,121770.0,-20709.0,40233.0,0.20,-14.53,33.04,13.70,163.0,-0.51,0.88,43.72,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,97.87,43.0,M-SC,2.89,85216.0,-15561.0,15654.0,0.48,-15.44,18.37,0.09,245.0,-0.99,0.61,11.95,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.27,220770.0,-48897.0,85239.0,0.01,-18.13,38.61,13.47,138.0,-0.57,1.59,11.66,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,34.0,H-SC,1.99,140990.0,-34913.0,76938.0,0.12,-19.85,54.57,23.89,149.0,-0.45,1.01,21.76,OX40N,NTT,CERAMICS
66,SIS,528.0,2079.27,39.0,H-SC,4.44,87950.0,-23082.0,46693.0,0.94,-20.79,53.09,21.26,156.0,-0.49,0.63,18.83,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,49.96,69.0,M-MC,10.56,236620.0,11658.0,156004.0,0.78,5.18,65.93,74.53,192.0,0.07,1.70,39.11,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,13.31,59.0,H-MC,3.86,195490.0,30820.0,12511.0,0.55,18.72,6.40,26.31,89.0,2.46,1.41,28.19,X40N,NTT,AC
84,VOLTAS,1530.00,-1.50,47.0,H-MC,3.07,206550.0,14808.0,22948.0,0.34,7.72,11.11,19.69,99.0,0.65,1.49,15.05,XY25,NTT,AC
17,CAMS,5211.76,-3.61,51.0,H-SC,1.81,109623.0,7619.0,36307.0,0.82,7.47,33.12,43.06,122.0,0.21,0.79,25.86,X40N,ATH,MISC
23,DABUR,735.00,-7.80,39.0,H-MC,6.85,249508.0,1440.0,106964.0,-0.65,0.58,42.87,43.70,102.0,0.01,1.79,15.11,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-45.32,54.0,H-MC,10.43,108920.0,8416.0,20695.0,-0.28,8.37,19.00,28.96,79.0,0.41,0.78,45.19,MH,ATH,METALS
11,BANKINDIA,190.00,-27.95,65.0,H-MC,7.89,185483.0,5675.0,106745.0,-0.32,3.16,57.55,62.52,88.0,0.05,1.33,38.32,XR,NTT,BANKS
85,WHIRLPOOL,2270.00,-44.33,33.0,M-SC,1.47,94012.0,2514.0,76234.0,-3.28,2.75,81.09,86.07,223.0,0.03,0.68,36.52,XR,NTT,DURABLES
37,INDIAMART,4850.92,-54.64,29.0,H-SC,10.47,126596.0,3260.0,130508.0,0.50,2.64,103.09,108.45,119.0,0.02,0.91,25.59,AR,ATH,MISC
86,WIPRO,420.00,-14.70,38.0,M-LC,5.50,149990.0,-955.0,110408.0,-1.09,-0.63,73.61,72.51,53.0,-0.01,1.08,5.35,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-16.46,33.0,H-LC,7.35,210716.0,-41052.0,94253.0,-2.55,-16.31,44.73,21.13,27.0,-0.44,1.52,13.34,X40,ATH,PAINTS
83,VBL,671.64,-19.44,34.0,H-LC,7.97,289939.0,-25903.0,138562.0,0.41,-8.20,47.79,35.67,5.0,-0.19,2.08,4.52,X40N,ATH,FMCG
42,ITC,452.00,-39.61,36.0,H-LC,2.11,194940.0,-5198.0,26083.0,-0.59,-2.60,13.38,10.44,4.0,-0.20,1.40,2.82,X40,NTT,FMCG
44,JIOFIN,387.00,-3.26,36.0,H-LC,13.14,262816.0,-4549.0,70014.0,-0.37,-1.70,26.64,24.48,48.0,-0.06,1.89,52.34,XY24,BTT,FINANCE
30,HCLTECH,1943.91,-1.94,39.0,H-LC,9.88,223505.0,-18419.0,81691.0,-1.30,-7.61,36.55,26.15,8.0,-0.23,1.61,5.62,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.49,47.0,H-LC,1.67,234652.0,-11499.0,90716.0,0.14,-4.67,38.66,32.18,16.0,-0.13,1.69,29.69,X200,ATH,IT
42,ITC,452.0,-39.61,36.0,H-LC,2.11,194940.0,-5198.0,26083.0,-0.59,-2.60,13.38,10.44,4.0,-0.20,1.40,2.82,X40,NTT,FMCG
65,SIEMENS,4671.5,-1.33,47.0,H-LC,2.82,158740.0,-27355.0,74830.0,-0.82,-14.70,47.14,25.51,15.0,-0.37,1.14,16.98,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.04,43.0,H-LC,4.29,214734.0,4368.0,24415.0,-0.47,2.08,11.37,13.68,37.0,0.18,1.54,18.94,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-10.51,47.0,H-LC,4.68,275207.0,9781.0,47005.0,-0.21,3.68,17.08,21.40,11.0,0.21,1.98,11.76,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.36,40.0,H-LC,12.12,285328.0,-60628.0,135474.0,-0.88,-17.52,47.48,21.64,1.0,-0.45,2.05,1.08,X40,ATH,IT
42,ITC,452.00,-39.61,36.0,H-LC,2.11,194940.0,-5198.0,26083.0,-0.59,-2.60,13.38,10.44,4.0,-0.20,1.40,2.82,X40,NTT,FMCG
83,VBL,671.64,-19.44,34.0,H-LC,7.97,289939.0,-25903.0,138562.0,0.41,-8.20,47.79,35.67,5.0,-0.19,2.08,4.52,X40N,ATH,FMCG
80,UNITDSPR,1644.00,-12.37,53.0,H-LC,7.11,229534.0,-4610.0,53229.0,-1.21,-1.97,23.19,20.77,86.0,-0.09,1.65,4.81,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.94,39.0,H-LC,9.88,223505.0,-18419.0,81691.0,-1.30,-7.61,36.55,26.15,8.0,-0.23,1.61,5.62,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-1.33,47.0,H-LC,2.82,158740.0,-27355.0,74830.0,-0.82,-14.70,47.14,25.51,15.0,-0.37,1.14,16.98,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.61,36.0,H-LC,2.11,194940.0,-5198.0,26083.0,-0.59,-2.60,13.38,10.44,4.0,-0.20,1.40,2.82,X40,NTT,FMCG
19,CIPLA,1795.00,-21.84,41.0,H-LC,4.89,208226.0,3726.0,37689.0,-1.18,1.82,18.10,20.25,10.0,0.10,1.50,10.71,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-16.46,33.0,H-LC,7.35,210716.0,-41052.0,94253.0,-2.55,-16.31,44.73,21.13,27.0,-0.44,1.52,13.34,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.04,43.0,H-LC,4.29,214734.0,4368.0,24415.0,-0.47,2.08,11.37,13.68,37.0,0.18,1.54,18.94,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.36,40.0,H-LC,12.12,285328.0,-60628.0,135474.0,-0.88,-17.52,47.48,21.64,1.0,-0.45,2.05,1.08,X40,ATH,IT
40,INFY,2275.00,-18.88,46.0,H-LC,8.09,316007.0,3119.0,168558.0,-0.74,1.00,53.34,54.87,3.0,0.02,2.27,7.71,X40,BTT,IT
42,ITC,452.00,-39.61,36.0,H-LC,2.11,194940.0,-5198.0,26083.0,-0.59,-2.60,13.38,10.44,4.0,-0.20,1.40,2.82,X40,NTT,FMCG
83,VBL,671.64,-19.44,34.0,H-LC,7.97,289939.0,-25903.0,138562.0,0.41,-8.20,47.79,35.67,5.0,-0.19,2.08,4.52,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,46.0,H-LC,8.30,250013.0,-11606.0,130832.0,-0.55,-4.44,52.33,45.57,7.0,-0.09,1.80,6.38,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6722.22,48.0,L-SC,5.36,74839.0,-18971.0,95592.0,-1.54,-20.22,127.73,81.67,269.0,-0.20,0.54,44.93,XR,NTT,CERAMICS
61,SAMMAANCAP,326.00,-171.59,57.0,M-SC,5.50,82662.0,-19548.0,112941.0,1.83,-19.13,136.63,91.37,208.0,-0.17,0.59,34.19,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-43.66,49.0,L-SC,42.92,83520.0,-30029.0,70082.0,-2.49,-26.45,83.91,35.27,268.0,-0.43,0.60,104.42,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-7.20,39.0,H-SC,11.43,86310.0,-14653.0,109182.0,-0.81,-14.51,126.50,93.63,148.0,-0.13,0.62,28.56,SR,ATH,CHEMICALS
49,MASFIN,398.61,-17.34,50.0,H-SC,12.68,94065.0,-3915.0,25520.0,-0.82,-4.00,27.13,22.05,152.0,-0.15,0.68,36.73,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1190.27,67.0,M-SC,11.40,175578.0,-5565.0,51041.0,-2.02,-3.07,29.07,25.10,235.0,-0.11,1.26,40.71,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-17.45,63.0,H-SC,16.30,221804.0,-48957.0,378908.0,-3.21,-18.08,170.83,121.86,133.0,-0.13,1.59,73.51,AR,ATH,IT
61,SAMMAANCAP,326.00,-171.59,57.0,M-SC,5.50,82662.0,-19548.0,112941.0,1.83,-19.13,136.63,91.37,208.0,-0.17,0.59,34.19,XY25,NTT,FINANCE
7,ATULAUTO,844.00,3678.38,52.0,M-SC,6.59,117764.0,-29604.0,81422.0,-1.71,-20.09,69.14,35.16,236.0,-0.36,0.85,20.28,XY24,NTT,AUTO
60,SAIL,228.00,49.96,69.0,M-MC,10.56,236620.0,11658.0,156004.0,0.78,5.18,65.93,74.53,192.0,0.07,1.70,39.11,XY24,BTT,STEEL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.24
1,25,43.71
2,50,74.83


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.91
LC    32.39
MC    23.69
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.25
X40      14.78
X40N     11.43
XR       11.22
XY25     10.13
AR        8.32
OX40N     7.86
X200      1.69
MH        1.65
X5K       1.44
SR        1.22
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.39
H-LC    25.95
H-MC    20.72
M-SC    13.02
M-LC     5.39
M-MC     2.67
L-SC     1.50
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.98,-5.48,40.29
IT,12.94,-16.76,80.42
FINANCE,10.07,-17.15,68.26
MISC,6.98,-18.29,84.05
BANKS,6.18,-16.14,77.24
PAINTS,5.64,-17.76,35.31
ELECTRICAL,5.48,-8.12,46.05
INSURANCE,3.76,-5.91,45.50
AC,3.60,3.16,13.05


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3172398.0,22
XR,1348795.0,14
AR,1173701.0,9
X40,795366.0,10
X40N,617632.0,9
OX40N,551784.0,10
XY25,430465.0,7
SR,251362.0,2
X5K,117353.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3704860.0,29
M-SC,1435634.0,17
H-MC,1225243.0,15
H-LC,1210888.0,15
M-LC,400086.0,4
M-MC,305909.0,2
L-SC,252451.0,3
L-MC,60494.0,1
L-LC,38471.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1238969.0      6
M-SC       XY24       821857.0      7
H-SC       AR         798931.0      5
           XR         782732.0      7
H-MC       XY24       563324.0      4
H-LC       X40        506059.0      5
H-SC       X40N       309306.0      4
M-MC       XY24       305909.0      2
H-SC       OX40N      259791.0      4
           SR         251362.0      2
H-LC       X40N       229480.0      3
H-MC       X40        215175.0      4
H-LC       AR         205662.0      2
M-LC       XY24       172325.0      2
H-MC       XY25       170096.0      2
M-SC       AR         169108.0      2
L-SC       XR         165674.0      2
M-SC       OX40N      134854.0      4
           XR         122742.0      2
M-LC       X5K        117353.0      1
M-SC       XY25       112941.0      1
M-LC       XR         110408.0      1
H-LC       XY25       108957.0      3
H-MC       XR         106745.0      1
H-LC       X200        90716.0      1
L-SC       OX40N       86777.0      1
H-MC       X40N        78846.0      2
M-SC       X40         74132.0      1
H-MC       OX40N       70362.0      1
H-LC       XY24        70014.0      1
H-SC       MH          63769.0      1
L-MC       XR          60494.0      1
L-LC       XY25        38471.0      1
H-MC       MH          20695.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
